# 새로운 데이터 예측

In [58]:
def getWeather(input_date):
    import requests
    import json
    from urllib.request import urlopen
    import pandas as pd
    from datetime import date, timedelta, datetime
    date = str(input_date)
    domain = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?"
    servicekey = "serviceKey=YlwyrgH2GoTFdpRhqVV1GWd7fsuSnU%2BUuUXMzkrGFZ8O9pfGNUXLgIxVuQ4LiEavT5CsUmjBcDZXTjSO0m0GVA%3D%3D&"
    pageno = "pageNo=1&"
    numofrows = "numOfRows=1000&"
    datatype = "dataType=JSON&"
    base_date = "base_date="+date+"&"
    base_time = "base_time=0500&"
    
    result_df = pd.DataFrame()
        
    # 데이터 리스트 생성
    list = {'x_': [60, 73, 92, 60, 69, 67, 63, 58, 50, 73, 91, 89, 102, 98, 52],
        'y_': [127, 134, 131, 121, 106, 100, 89, 74, 67, 66, 106, 90, 84, 76, 38],
       'area' : ['서울', '춘천', '강릉', '수원', '청주', '대전', '전주', '광주', '목포', '여수', '안동', '대구', '울산', '부산', '제주']}

    # 데이터프레임 생성
    list_ = pd.DataFrame(list)
    
    row_set = set()
    for _, row in list_.iterrows():
        x_ , y_, area = row[0], row[1], row[2]  
        nx = "nx=" + str(x_) + "&"
        ny = "ny=" + str(y_) + "&"
        
        url = domain + servicekey + pageno + numofrows + datatype + base_date + base_time + nx + ny
        res = requests.get(url)
        items = res.json().get('response').get('body').get('items').get('item')
        tmp = []
        avgt = ['0000', '0300', '0600', '0900', '1200', '1500', '1800', '2100']
        cate = ['TMP', 'WSD']
        for i in range(0, len(items), 12):
            td = dict()
            td['발효날짜'] = items[i].get('fcstDate')
            td['발효시각'] = items[i].get('fcstTime')
            for j in range(i, i+11):
                try:
                    if items[j].get('category') == 'TMP':
                        td['기온'] = items[j].get('fcstValue')
                    elif items[j].get('category') == 'WSD':
                        td['풍속'] = items[j].get('fcstValue')
                except:
                    pass
            tmp.append(td)
        df = pd.DataFrame(tmp)
        df = df.astype({'발효날짜':'datetime64' ,'기온':'float', '풍속':'float'})
        df2 = pd.DataFrame([df.groupby('발효날짜')['기온'].min(), df.groupby('발효날짜')['기온'].max()]).T
        df2.columns = ['최저기온', '최고기온']
        df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
           (df['발효시각'] == '1200') | (df['발효시각'] == '1500') | (df['발효시각'] == '1800') | (df['발효시각'] == '2100')].groupby('발효날짜')['기온', '풍속'].mean()
        df3.columns = ['평균기온', '평균풍속']
        df4 = pd.concat([df2, df3], axis=1)
        df2 = pd.DataFrame([df.groupby('발효날짜')['기온'].min(), df.groupby('발효날짜')['기온'].max()]).T
        df2.columns = ['최저기온', '최고기온']
        df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
       (df['발효시각'] == '1200') | (df['발효시각'] == '1500') | (df['발효시각'] == '1800') | (df['발효시각'] == '2100')].groupby('발효날짜')['기온', '풍속'].mean()
        df3.columns = ['평균기온', '평균풍속']
        df4 = pd.concat([df2, df3], axis=1).reset_index()
        
        # 
        X_new= df4[:3]
        X_new['ap_tmp'] = 13.12 + 0.6215 * X_new['평균기온'] - 11.37 * X_new['평균풍속']**0.16 + 0.3965 * X_new['평균기온'] * X_new['평균풍속']**0.16
        
        import pandas as pd
        from datetime import datetime
        import numpy as np
        
        start_date = datetime(2023, 1, 1)
        end_date = datetime(2023, 12, 31)
        
        # 날짜 범위 생성
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        # x 값 범위 설정
        x = np.linspace(0, len(date_range), len(date_range))
        # y 값 계산
        y = np.sin((2 * np.pi * x) / len(date_range))
        # 데이터프레임 생성
        df = pd.DataFrame({'x': date_range, 'y': y})
        
        X_new['sin'] = None
        
        # 각 행을 순회하며 조건과 비교하여 열 추가
        for index, row in X_new.iterrows():
            mask = df['x'] == row['발효날짜']
            if any(mask):  # 조건에 맞는 행이 하나 이상 있는지 확인
                X_new.loc[index, 'sin'] = df.loc[mask, 'y'].values[0]
            else:
                pass
            
        X_new.loc[abs(X_new['sin']) < 0.5 , 'sin'] = 0   
        X_new['dif'] = X_new['최저기온'] - X_new['최고기온']
        X_new.drop(['발효날짜', '최저기온', '최고기온', '평균기온', '평균풍속'], axis=1,inplace=True)
        X_new['area'] = area
        today = pd.to_datetime(str(input_date), format="%Y-%m-%d")
        tomorrow = today + timedelta(days=1)
        day_after_tomorrow = today + timedelta(days=2)
        l = [today, tomorrow, day_after_tomorrow]
        X_new['day'] = l
        
        result_df = pd.concat([result_df, X_new])

        

        
    return result_df

In [60]:
# 오늘 데이터와 지역 정보 넣어야함
X_live = getWeather(20230614)

C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:55: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:61: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['ap_tmp'] = 13.12 + 0.6215 *

C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:55: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:61: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['ap_tmp'] = 13.12 + 0.6215 *

C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:55: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:61: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['ap_tmp'] = 13.12 + 0.6215 *

C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:55: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:61: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['ap_tmp'] = 13.12 + 0.6215 *

C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:55: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:61: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df[(df['발효시각'] == '0000') | (df['발효시각'] == '0300') | (df['발효시각'] == '0600') | (df['발효시각'] == '0900') |
C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1053146375.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['ap_tmp'] = 13.12 + 0.6215 *

In [61]:
X_live

,ap_tmp,sin,dif,area,day
0,25.363231,0,-8.0,서울,2023-06-14
1,25.206567,0,-10.0,서울,2023-06-15
2,25.401896,0,-10.0,서울,2023-06-16
0,25.008149,0,-12.0,춘천,2023-06-14
1,24.072208,0,-10.0,춘천,2023-06-15
2,26.039832,0,-13.0,춘천,2023-06-16
0,22.740449,0,-5.0,강릉,2023-06-14
1,22.555977,0,-5.0,강릉,2023-06-15
2,26.933474,0,-11.0,강릉,2023-06-16
0,24.370098,0,-9.0,수원,2023-06-14


In [65]:
X_data = X_live[['area', 'day']]

In [66]:
X_data

,area,day
0,서울,2023-06-14
1,서울,2023-06-15
2,서울,2023-06-16
0,춘천,2023-06-14
1,춘천,2023-06-15
2,춘천,2023-06-16
0,강릉,2023-06-14
1,강릉,2023-06-15
2,강릉,2023-06-16
0,수원,2023-06-14


In [67]:
new_data = X_live.drop(['area', 'day'], axis=1)

In [68]:
new_data

,ap_tmp,sin,dif
0,25.363231,0,-8.0
1,25.206567,0,-10.0
2,25.401896,0,-10.0
0,25.008149,0,-12.0
1,24.072208,0,-10.0
2,26.039832,0,-13.0
0,22.740449,0,-5.0
1,22.555977,0,-5.0
2,26.933474,0,-11.0
0,24.370098,0,-9.0


In [62]:
import pickle

In [63]:
# K-means 모델 로드
with open('kmeans_model.pkl', 'rb') as f:
    km_model = pickle.load(f)

# 스케일러 로드
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [69]:
# 예측에 사용할 새로운 데이터 준비 (입력 데이터에 따라 적절하게 구성)

# 데이터 스케일링
scaled_data = scaler.transform(new_data)

# K-means 모델을 사용하여 예측 수행
cluster_labels = km_model.predict(scaled_data)

# 예측된 클러스터 레이블 출력
print(cluster_labels)

[0 0 0 0 0 0 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 7 0 0 0 0 0 7 0]


C:\Users\yyseo\anaconda3\envs\ml\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [72]:
X_data['label']= cluster_labels

C:\Users\yyseo\AppData\Local\Temp\ipykernel_17552\1207299785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_data['label']= cluster_labels


In [73]:
X_data

,area,day,label
0,서울,2023-06-14,0
1,서울,2023-06-15,0
2,서울,2023-06-16,0
0,춘천,2023-06-14,0
1,춘천,2023-06-15,0
2,춘천,2023-06-16,0
0,강릉,2023-06-14,7
1,강릉,2023-06-15,7
2,강릉,2023-06-16,0
0,수원,2023-06-14,0
